<a href="https://colab.research.google.com/github/Paul-NIROB/agentic-ai/blob/main/Fine_tuning_Lab_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers accelerate evaluate


In [ ]:
!pip install -q transformers datasets evaluate accelerate


In [ ]:
!pip install -q evaluate
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import evaluate

In [ ]:
dataset = load_dataset("ag_news")
dataset


In [ ]:
model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4
)


In [ ]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(5000))
small_test = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

trainer.train_dataset = small_train
trainer.eval_dataset = small_test


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


# Fine-Tuning a Small Language Model (SLM)

## Agentic AI – Lab Task 1

### Objective
The objective of this task is to fine-tune a Small Language Model (SLM) with fewer than 3 billion parameters using a text dataset from Hugging Face and evaluate its performance.

### Tools Used
- Google Colab
- Hugging Face Transformers
- Hugging Face Datasets
- DistilBERT model


## Dataset Description

The AG News dataset is a text classification dataset consisting of news articles categorized into four classes:
- World
- Sports
- Business
- Science/Technology

The dataset is divided into training and testing splits and is suitable for text classification tasks.


## Results and Observations

The fine-tuned DistilBERT model achieved strong performance on the AG News dataset
with high classification accuracy. The training loss decreased steadily, showing
effective learning during fine-tuning.

Using a reduced dataset significantly decreased training time while maintaining
acceptable performance, making this approach suitable for resource-constrained
environments.



## Conclusion

This lab demonstrated the fine-tuning of a Small Language Model with fewer than
3 billion parameters using a Hugging Face dataset. The results show that SLMs can
be efficiently adapted to downstream tasks using limited computational resources.
